In [4]:
import pickle
import torch
import torch.optim as optim
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import data_loader
import models
import pandas as pd

ncsr = pd.read_csv('../justage_vars_init.csv', index_col=0)
pkl = open('../gad_ncsr_for_ddp.pickle', "rb")
dpp_data = {}
dpp_data['ncsr'] = pickle.load(pkl)
ddp_data = dpp_data
pkl.close()

max_len = max([len(ddp_data['ncsr'][x]) for  x in range(len(ddp_data['ncsr']))])
n_event_type = dim_process = len(ncsr.columns) 
n_sample = len(ddp_data['ncsr'])
context_dim = 1


train_input = data_loader.process_seq(ddp_data, list(range(n_sample)), max_len=max_len, n_event_type=n_event_type, tag_batch = 'ncsr', dtype=np.float32)

batch_input_np = list(train_input)
df_patient_static_mat = np.ones((1, n_sample)).astype('float32')
batch_input_np.append(df_patient_static_mat)


gap = batch_input_np[0][:-1, :] - batch_input_np[0][1:, :]
gap_mean = np.mean(gap)
gap_std = np.std(gap)

ddp_model = pickle.load(open('../gad_ddp_1_model.pkl', 'rb'))

In [98]:
max_len = max([len(ddp_data['ncsr'][x]) for  x in range(len(ddp_data['ncsr']))])
n_event_type = dim_process = len(ncsr.columns) 
n_sample = len(ddp_data['ncsr'])
context_dim = 1
test_data = {}
test_data['ncsr'] = []
test_data['ncsr'].append(ddp_data['ncsr'][8][0:1])


tst = data_loader.process_seq(test_data, list(range(1)), max_len=max_len, n_event_type=n_event_type, tag_batch = 'ncsr', dtype=np.float32)
batch_input_np = list(tst)

n_record = tst[0].shape[1]

df_patient_static_mat = np.ones((1, n_sample)).astype('float32')
batch_input_np.append(df_patient_static_mat)
tst = batch_input_np
idx = np.random.choice(list(range(n_record)), size=1, replace=False)
mini_batch = list(map(lambda x: torch.tensor(x[..., idx]), tst))
ddp_model.set_input(*mini_batch)

In [104]:
with torch.no_grad():
            ddp_model.update_state_given_event(torch.tensor([ 0.16901408450704225]), torch.tensor([287]))
            #ddp_model.forward()
            x = ddp_model.get_prediction()
            y = ddp_model.next_event_time(100)

In [92]:
ddp_data['ncsr'][8][3]

{'idx_event': 4,
 'type_event': 287,
 'time_since_start': 0.16901408450704225,
 'time_since_last_same_event': 0.16901408450704225,
 'time_since_last_event': 0.11267605633802816}

ddp_model.get_prediction() dim: 309x126x2

get_prediction:  symptom -> timepoint -> case


In [69]:
chawkes = models.CHawkes(
    n_event_type=n_event_type, 
    n_context_dim=context_dim, 
    first_occurrence_only=False, 
    #alpha_mat_np=alpha_init,
    #lambda_mat_np=lambda_init,
)

In [71]:
chawkes.set_input(*mini_batch)
chawkes.forward()
chawkes.get_likelihood_seq()

tensor([-6598.0791], grad_fn=<SubBackward0>)

In [174]:
for idx, x in enumerate(case_batch):
    case_batch[idx][x!=0]  =  0

In [175]:
case_batch[8][0] = torch.Tensor([np.float32(1.0)]).type(torch.FloatTensor)

In [227]:
type(case_batch)

list